In [ ]:
!nvidia-smi

Sun May 29 21:19:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers==4.19.1
!pip install sentencepiece==0.1.96

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.3 MB/s 
     |████████████████████████████████| 6.6 MB 49.4 MB/s 
     |████████████████████████████████| 596 kB 75.4 MB/s 
     |████████████████████████████████| 86 kB 6.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 4.2 MB/s 


In [ ]:
# ----------------------------------------------
# Load Libraries
# ----------------------------------------------
import pathlib
from pathlib import Path
import sys
import os
import math
import random
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from transformers import AdamW
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import BertForSequenceClassification, BertConfig, BertModel
from transformers import get_cosine_schedule_with_warmup

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split

import gc
gc.enable()

# Config

In [ ]:
class CFG:
    exp_id = 'exp039'
    input_path = 'input/'
    cpc_path = 'input/'
    model_path = 'anferico/bert-for-patents'
    out_base = 'output'
    out_path = f'{out_base}/{exp_id}'
    scores_path = f'{out_path}/scores'
    
    debug = False
    fold1_only = False
    upload_dataset = True
    debug_size = 100
    log_interval = 1822 # 未使用
    seed = 42
    max_len = 92
    learning_rate = 2e-5
    weight_decay = 0.01
    num_classes = 1
    num_fold = 4
    epochs = 5
    batch_size = 16
    device = "cuda" if torch.cuda.is_available() else "cpu"

    if debug:
      epochs = 1
      upload_dataset = False

    if fold1_only:
      upload_dataset = False

In [ ]:
!mkdir -p {CFG.out_path}
!mkdir -p {CFG.scores_path}

# Preproc

In [ ]:
# ----------------------------------------------
# Set SEED
# ----------------------------------------------
# seed
SEED = CFG.seed
def set_seed(SEED):
    random.seed(SEED)
    np.random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    
set_seed(SEED)

In [ ]:
train_df = pd.read_csv(f"{CFG.input_path}train.csv")
# https://www.kaggle.com/code/gauravbrills/folds-dump-the-two-paths-fix
cpc_texts = torch.load(CFG.cpc_path+"cpc_texts_fixed.pth")
train_df['context_text'] = train_df['context'].map(cpc_texts)
display(train_df.head())

,id,anchor,target,context,score,context_text
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...


In [ ]:
!pip install -q iterative-stratification
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

dfx = pd.get_dummies(train_df, columns=["score"]).groupby(["anchor"], as_index=False).sum()
cols = [c for c in dfx.columns if c.startswith("score_") or c == "anchor"]
dfx = dfx[cols]

mskf = MultilabelStratifiedKFold(n_splits=CFG.num_fold, shuffle=True, random_state=42)
labels = [c for c in dfx.columns if c != "anchor"]
dfx_labels = dfx[labels]
dfx["fold"] = -1

for fold, (trn_, val_) in enumerate(mskf.split(dfx, dfx_labels)):
    print(len(trn_), len(val_))
    dfx.loc[val_, "fold"] = fold

train_df = train_df.merge(dfx[["anchor", "fold"]], on="anchor", how="left")

550 183
549 184
550 183
550 183


In [ ]:
train_df['fold'].value_counts().sort_index()

0    9379
1    8860
2    8612
3    9622
Name: fold, dtype: int64

In [ ]:
train_df.groupby('fold')['score'].value_counts().sort_index()

fold  score
0     0.00     1854
      0.25     2969
      0.50     3238
      0.75      998
      1.00      320
1     0.00     1786
      0.25     2790
      0.50     2997
      0.75     1010
      1.00      277
2     0.00     1836
      0.25     2641
      0.50     2883
      0.75      980
      1.00      272
3     0.00     1995
      0.25     3119
      0.50     3182
      0.75     1041
      1.00      285
Name: score, dtype: int64

In [ ]:
print(train_df.shape)
train_df.head()

(36473, 7)


,id,anchor,target,context,score,context_text,fold
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,0
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,0
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,0
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,0
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,0


In [ ]:
train_df['input'] = train_df['anchor'] + '[SEP]' + train_df['target'] + '[SEP]' + train_df['context_text']

In [ ]:
train_df.head()

,id,anchor,target,context,score,context_text,fold,input
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,0,abatement[SEP]abatement of pollution[SEP]HUMAN...
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,0,abatement[SEP]act of abating[SEP]HUMAN NECESSI...
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,0,abatement[SEP]active catalyst[SEP]HUMAN NECESS...
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,0,abatement[SEP]eliminating process[SEP]HUMAN NE...
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,0,abatement[SEP]forest region[SEP]HUMAN NECESSIT...


# Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(CFG.model_path)

Downloading:   0%|          | 0.00/327 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/322k [00:00<?, ?B/s]

# Dataset

In [ ]:
def prepare_input(tokenizer, text):
    inputs = tokenizer(text,
                           add_special_tokens=True,
                           max_length=CFG.max_len,
                           padding="max_length",
                           truncation=True,
                           return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class TrainDataset(Dataset):
    def __init__(self, df):
        self.inputs = df['input'].values
        self.label = df['score'].values

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, item):
        inputs = self.inputs[item]
        label = self.label[item]
        outputs = prepare_input(tokenizer, inputs)
        outputs['label'] = torch.tensor(label, dtype=torch.float32)

        return outputs

# Train

In [ ]:
# ----------------------------------------------
# Model
# ----------------------------------------------
class AttentionHead(nn.Module):
    def __init__(self, in_features, hidden_dim, num_targets):
        super().__init__()
        self.in_features = in_features
        self.middle_features = hidden_dim
        self.W = nn.Linear(in_features, hidden_dim)
        self.V = nn.Linear(hidden_dim, 1)
        self.out_features = hidden_dim

    def forward(self, features):
        att = torch.tanh(self.W(features))
        score = self.V(att)
        attention_weights = torch.softmax(score, dim=1)
        context_vector = attention_weights * features
        context_vector = torch.sum(context_vector, dim=1)

        return context_vector

class PPPMModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.config = AutoConfig.from_pretrained(CFG.model_path)
        self.config.attention_probs_dropout_prob = 0.0
        self.config.hidden_dropout_prob = 0.0
        self.pre_model = AutoModel.from_pretrained(CFG.model_path, config=self.config)
        self.head = AttentionHead(self.config.hidden_size, self.config.hidden_size,1)
        #self.dropout = nn.Dropout(0.3)
        self.regressor = nn.Linear(self.config.hidden_size, CFG.num_classes)
    
    def forward(self, inputs):
        pre_out = self.pre_model(**inputs)
        x = pre_out[0]
        x = self.head(x)
        x = self.regressor(x)
        return x

In [ ]:
"""ds = TrainDataset(train_df.head(20))
model = PPPMModel().to(CFG.device)
train_loader = DataLoader(ds,
                        batch_size=CFG.batch_size,
                        shuffle=True,
                        drop_last=True,
                        num_workers=2)
ds_l = list(train_loader)

with torch.no_grad():
    for batch_idx, data in enumerate(train_loader):
        input_ids = data['input_ids'].to(CFG.device)
        token_type_ids = data['token_type_ids'].to(CFG.device)
        attention_mask = data['attention_mask'].to(CFG.device)
        label = data['label'].to(CFG.device)
        model = model.to(CFG.device)
        
        output = model(input_ids, token_type_ids, attention_mask)"""

"ds = TrainDataset(train_df.head(20))\nmodel = PPPMModel().to(CFG.device)\ntrain_loader = DataLoader(ds,\n                        batch_size=CFG.batch_size,\n                        shuffle=True,\n                        drop_last=True,\n                        num_workers=2)\nds_l = list(train_loader)\n\nwith torch.no_grad():\n    for batch_idx, data in enumerate(train_loader):\n        input_ids = data['input_ids'].to(CFG.device)\n        token_type_ids = data['token_type_ids'].to(CFG.device)\n        attention_mask = data['attention_mask'].to(CFG.device)\n        label = data['label'].to(CFG.device)\n        model = model.to(CFG.device)\n        \n        output = model(input_ids, token_type_ids, attention_mask)"

In [ ]:
# ----------------------------------------------
# func: valid, predict
# ----------------------------------------------
def predict(model, dataloader):
    model.eval()
    result = np.zeros((len(dataloader.dataset), CFG.num_classes))
    idx = 0
    
    with torch.no_grad():
        for batch_idx, data in enumerate(dataloader):
          inputs = {}
          for k, v in data.items():
            inputs[k] = v.to(CFG.device)            
          output = model(inputs)
          result[idx:idx + output.shape[0], :] = output.to('cpu')
          idx += output.shape[0]
            
    return result


def valid_mse(model, dataloader):
    model.eval()
    result = np.zeros((len(dataloader.dataset), CFG.num_classes))
    idx = 0
    mse_sum = 0
    
    with torch.no_grad():
        for batch_idx, data in enumerate(dataloader):
          inputs = {}
          for k, v in data.items():
            if k != 'label':
              inputs[k] = v.to(CFG.device)            
          label = data['label'].to(CFG.device)          
          output = model(inputs)
          result[idx:idx + output.shape[0], :] = output.to('cpu')
          idx += output.shape[0]

          mse_sum += nn.MSELoss(reduction='sum')(output.flatten(), label).item()
            
    return mse_sum/(len(dataloader.dataset)), result.reshape(len(result))


def metric_pearson(predictions, labels):
    pearson = np.corrcoef(predictions, labels)[0][1]       
    return pearson

In [ ]:
# ----------------------------------------------
# func: train
# ----------------------------------------------
def train_fn(
    model,
    save_path,
    train_loader,
    val_loader,
    optimizer,
    scheduler=None,
    num_epochs=CFG.epochs
):

    best_score = 0
    best_epoch = 0
    log_interval = CFG.log_interval
    accu_step = 0
    oof_preds = None

    start = time.time()

    for epoch in range(num_epochs):
        val_score = None

        for batch_idx, data in enumerate(train_loader):
          inputs = {}
          for k, v in data.items():
            if k != 'label':
              inputs[k] = v.to(CFG.device)            
          label = data['label'].to(CFG.device)

          optimizer.zero_grad()
          model.train()

          output = model(inputs)
          loss = nn.MSELoss()(output.flatten(), label)

          loss.backward()
          optimizer.step()

          if scheduler:
              scheduler.step()
          if CFG.debug == True:
            if (accu_step > 0) & (accu_step % CFG.debug_size == 0):
                break
          accu_step += 1

        val_start = time.time()
        val_score, predictions = valid_mse(model, val_loader)
        pearson = metric_pearson(predictions, val_loader.dataset.label)
        print(f"Epoch {epoch+1}, Step {batch_idx+1}, train_loss: {loss:0.5f}, val_loss: {val_score:0.5f}, pearson: {pearson:0.5f}")
        if pearson > best_score:
            print(f"Model Inproved: {best_score} ----> {pearson}")
            best_score = pearson
            oof_preds = predictions
            torch.save(model.state_dict(), save_path)
        print(f"validation elasped time: {time.time() - val_start: 0.3}")

    print(f"total elasped time: {time.time() - start: 0.3}")
    start = time.time()

    return best_score, oof_preds

# ----------------------------------------------
# create optimizer
# ----------------------------------------------
def create_optimizer(model):
    named_params = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optim_params = []
    for idx_, (name_, params_) in enumerate(named_params):
        weight_decay = 0 if name_ in no_decay else 0.01
        optim_params.append({'params':params_,
                            'weight_decay': weight_decay,
                            })

    return AdamW(optim_params)

In [ ]:
# ----------------------------------------------
# Main Loop
# ----------------------------------------------
val_scores = []
oof_df = pd.DataFrame()

for fold in range(CFG.num_fold): 
    print(f"*** FOLD {fold+1} / {CFG.num_fold}***")

    save_path = f"{CFG.out_path}/model_{fold+1}.pth"

    train_data = train_df[train_df['fold'] != fold]
    valid_data = train_df[train_df['fold'] == fold]
    train_set = TrainDataset(train_data)
    valid_set = TrainDataset(valid_data)

    train_loader = DataLoader(train_set,
                            batch_size=CFG.batch_size,
                            shuffle=True,
                            drop_last=True,
                            num_workers=2)
    valid_loader = DataLoader(valid_set,
                            batch_size=CFG.batch_size,
                            shuffle=False,
                            drop_last=False,
                            num_workers=2)

    model = PPPMModel().to(CFG.device)
    optimizer = AdamW(model.parameters(), lr=CFG.learning_rate)
    scheduler = get_cosine_schedule_with_warmup(
        optimizer,
        num_training_steps=CFG.epochs*len(train_loader),
        num_warmup_steps=100
    )

    val_score, val_preds = train_fn(model, save_path, train_loader, valid_loader, optimizer, scheduler=scheduler)

    val_scores.append(val_score)
    valid_data['preds'] = val_preds
    oof_df = pd.concat([oof_df, valid_data])

    del model
    torch.cuda.empty_cache()

    print(val_scores)
    print("Mean:", np.array(val_scores).mean())
    if CFG.fold1_only == True:
        if fold == 0:
            break

*** FOLD 1 / 4***


Downloading:   0%|          | 0.00/1.29G [00:00<?, ?B/s]

Some weights of the model checkpoint at anferico/bert-for-patents were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of Adam

Epoch 1, Step 1693, train_loss: 0.01336, val_loss: 0.02582, pearson: 0.80886
Model Inproved: 0 ----> 0.8088557806553889
validation elasped time:  90.5
Epoch 2, Step 1693, train_loss: 0.01616, val_loss: 0.02163, pearson: 0.82345
Model Inproved: 0.8088557806553889 ----> 0.8234484076534351
validation elasped time:  90.2
Epoch 3, Step 1693, train_loss: 0.00536, val_loss: 0.02153, pearson: 0.82263
validation elasped time:  84.7
Epoch 4, Step 1693, train_loss: 0.00126, val_loss: 0.02174, pearson: 0.82525
Model Inproved: 0.8234484076534351 ----> 0.8252490822615616
validation elasped time:  90.0
Epoch 5, Step 1693, train_loss: 0.00084, val_loss: 0.02177, pearson: 0.82404
validation elasped time:  84.7
total elasped time:  4.51e+03
[0.8252490822615616]
Mean: 0.8252490822615616
*** FOLD 2 / 4***


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
Some weights of the model checkpoint at anferico/bert-for-patents were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTrai

Epoch 1, Step 1725, train_loss: 0.03418, val_loss: 0.02827, pearson: 0.78290
Model Inproved: 0 ----> 0.7829010162346098
validation elasped time:  86.0
Epoch 2, Step 1725, train_loss: 0.00725, val_loss: 0.02432, pearson: 0.80464
Model Inproved: 0.7829010162346098 ----> 0.8046384887417994
validation elasped time:  85.6
Epoch 3, Step 1725, train_loss: 0.00218, val_loss: 0.02295, pearson: 0.81284
Model Inproved: 0.8046384887417994 ----> 0.8128366247995321
validation elasped time:  85.6
Epoch 4, Step 1725, train_loss: 0.00391, val_loss: 0.02412, pearson: 0.80699
validation elasped time:  80.1
Epoch 5, Step 1725, train_loss: 0.00113, val_loss: 0.02362, pearson: 0.80824
validation elasped time:  80.3
total elasped time:  4.56e+03
[0.8252490822615616, 0.8128366247995321]
Mean: 0.8190428535305468
*** FOLD 3 / 4***


Some weights of the model checkpoint at anferico/bert-for-patents were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 1, Step 1741, train_loss: 0.03406, val_loss: 0.02705, pearson: 0.80101
Model Inproved: 0 ----> 0.8010090388675994
validation elasped time:  83.3
Epoch 2, Step 1741, train_loss: 0.01193, val_loss: 0.02713, pearson: 0.82635
Model Inproved: 0.8010090388675994 ----> 0.8263451025025227
validation elasped time:  83.8
Epoch 3, Step 1741, train_loss: 0.01952, val_loss: 0.02200, pearson: 0.82681
Model Inproved: 0.8263451025025227 ----> 0.8268050236955292
validation elasped time:  83.7
Epoch 4, Step 1741, train_loss: 0.00357, val_loss: 0.02127, pearson: 0.83191
Model Inproved: 0.8268050236955292 ----> 0.831910018209814
validation elasped time:  84.1
Epoch 5, Step 1741, train_loss: 0.00105, val_loss: 0.02128, pearson: 0.83147
validation elasped time:  77.9
total elasped time:  4.6e+03
[0.8252490822615616, 0.8128366247995321, 0.831910018209814]
Mean: 0.8233319084236359
*** FOLD 4 / 4***


Some weights of the model checkpoint at anferico/bert-for-patents were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 1, Step 1678, train_loss: 0.00937, val_loss: 0.02418, pearson: 0.79700
Model Inproved: 0 ----> 0.7970035151103974
validation elasped time:  93.1
Epoch 2, Step 1678, train_loss: 0.02136, val_loss: 0.02428, pearson: 0.79939
Model Inproved: 0.7970035151103974 ----> 0.7993937473115227
validation elasped time:  92.9
Epoch 3, Step 1678, train_loss: 0.00836, val_loss: 0.02349, pearson: 0.80295
Model Inproved: 0.7993937473115227 ----> 0.802948404577044
validation elasped time:  93.1
Epoch 4, Step 1678, train_loss: 0.00339, val_loss: 0.02362, pearson: 0.80681
Model Inproved: 0.802948404577044 ----> 0.8068099831319564
validation elasped time:  93.3
Epoch 5, Step 1678, train_loss: 0.00449, val_loss: 0.02349, pearson: 0.80516
validation elasped time:  87.3
total elasped time:  4.5e+03
[0.8252490822615616, 0.8128366247995321, 0.831910018209814, 0.8068099831319564]
Mean: 0.819201427100716


In [ ]:
"""for batch_idx, data in enumerate(valid_loader):
  print(batch_idx)
"""

'for batch_idx, data in enumerate(valid_loader):\n  print(batch_idx)\n'

In [ ]:
scores = {f'fold{i}':j for i,j in enumerate(val_scores)}
scores['oof'] = np.corrcoef(oof_df['preds'], oof_df['score'])[0][1]
scores = pd.Series(scores)
print(scores)

fold0    0.825249
fold1    0.812837
fold2    0.831910
fold3    0.806810
oof      0.818797
dtype: float64


In [ ]:
scores.to_csv(f'{CFG.out_path}/scores.csv')
oof_df.to_csv(f'{CFG.out_path}/oof_df.csv')